In [1]:
import os

In [2]:
%pwd

'c:\\Users\\latif\\OneDrive\\Belgeler\\GitHub\\Text_Summarization\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\latif\\OneDrive\\Belgeler\\GitHub\\Text_Summarization'